In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [ ]:
data_llama3 = {
    'Metric': ['Class Compliance', 'Class Coverage', 'Property Compliance', 'Property Coverage'],
    'OG': [83.3, 50.0, 80.0, 66.7],
    'AG': [100.0, 50.0, 88.9, 100.0],
}
